In [6]:
!pip install pandas feedparser newspaper3k nltk requests lxml_html_clean

import feedparser
import pandas as pd
import time
import requests
import calendar
from datetime import datetime
from newspaper import Article, Config
import nltk


try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


YEAR = 2024
target_months = range(1, 13) # 1월부터 12월까지 프로젝트1번이랑 같이 1년치만 하기로
all_data = []

# 이건 구글링한건데 차단 방지용 헤더 였던걸로 기억함
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10


for month in target_months:
    last_day = calendar.monthrange(YEAR, month)[1]
    start_str = f"{YEAR}-{month:02d}-01"
    end_str = f"{YEAR}-{month:02d}-{last_day}"

    # NVIDIA + (주식 or 실적 or AI)
    query = f"NVIDIA (stock OR earnings OR AI) after:{start_str} before:{end_str}"

    rss_url = (
        "https://news.google.com/rss/search?"
        f"q={query.replace(' ', '+')}"
        "&hl=en-US&gl=US&ceid=US:en"
    )

    print(f"\n[{month}월] 수집 시작... ({start_str} ~ {end_str})")

    feed = feedparser.parse(rss_url)
    total_entries = len(feed.entries)
    print(f"    발견된 기사: {total_entries}개 -> 전량 수집 시도")

    if total_entries == 0:
        continue

    count = 0
    for entry in feed.entries:
        try:
            title = entry.title
            original_link = entry.link
            published_date = datetime(*entry.published_parsed[:6]).strftime("%Y-%m-%d")

            try:
                res = requests.get(original_link, headers={'User-Agent': user_agent}, timeout=5)
                final_url = res.url
            except:
                continue # 링크가 깨졌으면 패스

            # 본문 추출
            article = Article(final_url, config=config)
            article.download()
            article.parse()
            article.nlp()

            # 저장
            all_data.append([
                published_date,
                title,
                article.summary,
                article.text,
                final_url
            ])
            count += 1

            # 진행 상황 표시 (10개마다 점 찍기)
            if count % 10 == 0:
                print(f"    ...{count}개 완료", end='\r')

            # 계속 오류나서 찾아보니 sleep 안 넣어줌 차단된다고.....
            time.sleep(0.5)

        except Exception as e:
            pass

    print(f"\n {month}월 최종 완료: {count}개 / {total_entries}개")
    time.sleep(1)


df = pd.DataFrame(all_data, columns=["date", "title", "summary", "content", "url"])
df = df.drop_duplicates(subset="title")
df = df.sort_values(by="date")

# 키워드 태깅 함수
KEYWORDS = {
    "AI": ["ai", "gpu", "intelligence"],
    "EARNINGS": ["earnings", "revenue", "profit", "sales"],
    "STOCK": ["stock", "shares", "market", "bull", "bear"],
    "RISK": ["risk", "drop", "fall", "concern"]
}

def tag_keywords(text):
    if not text: return ""
    text = text.lower()
    tags = []
    for key, words in KEYWORDS.items():
        if any(w in text for w in words):
            tags.append(key)
    return ",".join(tags)

df["keywords"] = df["title"].apply(tag_keywords)

filename = f"nvidia_2024_full_unlimited.csv"
df.to_csv(filename, index=False, encoding="utf-8-sig")

print(f"\n 총 {len(df)}개의 기사가 '{filename}'에 저장되었습니다.")


[1월] 수집 시작... (2024-01-01 ~ 2024-01-31)
    발견된 기사: 100개 -> 전량 수집 시도

 1월 최종 완료: 100개 / 100개

[2월] 수집 시작... (2024-02-01 ~ 2024-02-29)
    발견된 기사: 100개 -> 전량 수집 시도

 2월 최종 완료: 100개 / 100개

[3월] 수집 시작... (2024-03-01 ~ 2024-03-31)
    발견된 기사: 100개 -> 전량 수집 시도

 3월 최종 완료: 100개 / 100개

[4월] 수집 시작... (2024-04-01 ~ 2024-04-30)
    발견된 기사: 100개 -> 전량 수집 시도

 4월 최종 완료: 100개 / 100개

[5월] 수집 시작... (2024-05-01 ~ 2024-05-31)
    발견된 기사: 100개 -> 전량 수집 시도

 5월 최종 완료: 100개 / 100개

[6월] 수집 시작... (2024-06-01 ~ 2024-06-30)
    발견된 기사: 100개 -> 전량 수집 시도

 6월 최종 완료: 100개 / 100개

[7월] 수집 시작... (2024-07-01 ~ 2024-07-31)
    발견된 기사: 100개 -> 전량 수집 시도

 7월 최종 완료: 100개 / 100개

[8월] 수집 시작... (2024-08-01 ~ 2024-08-31)
    발견된 기사: 100개 -> 전량 수집 시도

 8월 최종 완료: 100개 / 100개

[9월] 수집 시작... (2024-09-01 ~ 2024-09-30)
    발견된 기사: 100개 -> 전량 수집 시도

 9월 최종 완료: 100개 / 100개

[10월] 수집 시작... (2024-10-01 ~ 2024-10-31)
    발견된 기사: 100개 -> 전량 수집 시도

 10월 최종 완료: 100개 / 100개

[11월] 수집 시작... (2024-11-01 ~ 2024-11-30)
    발견된 기사: 100개